In [ ]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Nonlinear constraints

In [ ]:
from angle_set import AngleSet

d = 2 # do not change.
N = 4

#np.random.seed(51) # non-convex
np.random.seed(7) #convex

angle_set = AngleSet(N=N, d=d)
angle_set.set_points(mode='random')
angle_set.plot_all()

In [ ]:
from simulation_discrepancy import get_noisy
from algorithms import constraint_sine
from angle_set import get_linear_constraints
from scipy.optimize import minimize

linear = True
np.random.seed(1)
noisy_angles = get_noisy(angle_set.theta, scale=0.2)

chosen_quad = [0, 1, 2, 3]
cost0 = constraint_sine(noisy_angles, angle_set.corners, chosen_quad)
print('cost0', cost0)


cons = []

print('adding non-linear constraints')
cons.append({
    'type': 'eq',
    'fun': lambda x: constraint_sine(x, angle_set.corners, chosen_quad)
})
if linear:
    print('adding linear constraints')
    Afull, bfull = get_linear_constraints(angle_set)
    cons.append({
        'type': 'eq',
        'fun': lambda x: Afull.dot(x) - bfull
    })
options = {
    'disp': False,
    'ftol': 1e-12, 
    'maxiter': 400,
}
loss = lambda x: 0.5*np.linalg.norm(x - noisy_angles)**2
res = minimize(loss, x0=noisy_angles, constraints=cons,
               method='SLSQP', options=options)
denoised_angles = res.x
if not res.success:
    print('Warning: did not solve.')
cost1 = constraint_sine(denoised_angles, angle_set.corners, chosen_quad)
print('cost1', cost1)

In [ ]:
from algorithms import get_angles
from math import cos, sin

# those are the angles we have imposed constraints for.
print('constrained imposed on', chosen_quad)
g, h, i, j, k, l = get_angles(denoised_angles, angle_set.corners, chosen_quad)
print('should be equal: {:.2e} - {:.2e} = {:.2e}'.format(sin(i) * sin(h) * sin(l), sin(g) * sin(j) * sin(k), 
                                                     sin(i) * sin(h) * sin(l) - sin(g) * sin(j) * sin(k)))

for shift in range(1, 4):
    shifted_quad = np.roll(chosen_quad, shift=shift)
    a, b, c, d, e, f = get_angles(denoised_angles, angle_set.corners, shifted_quad)
    print(f'{shifted_quad}: not necessarily equal', sin(c) * sin(b) * sin(f), sin(a) * sin(d) * sin(e))

Next we kind of implicitly impose a few linear constraints, because we 
express the adjacent set of angles in terms of the original angles, 
thus using the "single" constraints around the points. 
Note that this equation only holds for a convex set up where the 
points 0 to 3 are in clockwise order (seed 7 gives such a setup).
this is not a limiting factor, generalizing would just make the code
a bit more ugly.

In [ ]:
print('should be equal: {:.2e} - {:.2e} = {:2e}'.format(sin(g+h) * sin(g+i) * sin(l), sin(j-l) * sin(k+l) * sin(g),
                                                   sin(g+h) * sin(g+i) * sin(l) - sin(j-l) * sin(k+l) * sin(g)))
print('should be equal: {:.2e} - {:.2e} = {:2e}'.format(sin(g+h) * sin(j+h) * sin(k), sin(i-k) * sin(l+k) * sin(h),
                                                   sin(g+h) * sin(j+h) * sin(k) - sin(i-k) * sin(l+k) * sin(h)))

In [ ]:
up=cos(h)**2 + sin(h)/sin(j)*cos(j)*cos(h) + sin(h)/sin(g)*cos(h)*cos(g) + sin(h)**2 / (sin(g)*sin(j))*cos(g)*cos(j)
down=cos(k)**2 + sin(k)/sin(l)*cos(l)*cos(k) - sin(k)/sin(i)*cos(k)*cos(i) - sin(k)**2 / (sin(i)*sin(l))*cos(i)*cos(l)
print(up, down, up-down)

## Findings

When we do not impose the linear constraints, then imposing sine constraints on one choice of angles does not imply the constraints on the adjacent set of angles. The constraint is almost satisfied, but not exactly, and the error does not seem to be due to numerical problems only. To verify this, set linear to False above and see that the last "should be equal" lines are not equal.

When we impose the linear constraints, then imposing on constraint also implies all others.